In [1]:
#Importation des differentes packages
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re

In [36]:
#Importation des données de metagenomique en un dataframe pandas
df = pd.read_csv("1335.vdb.tab", sep ='\t')

#Expression reguliere pour recuperer valeur entre  parentheses
pattern='\((\d+)\)'

#Découpage de la colonne 'OTUConTaxonomy'
sub_dtf=df['OTUConTaxonomy'].map(lambda x: re.sub(pattern, '', x).rstrip(";")).str.split(";",expand= True)
df[['regne','embranchement','classe','ordre', 'famille', 'genre']] = sub_dtf

#inutile car les noms de bcteries sont composé du genre et de l'espece
# df['name']= df['famille']+' '+df['genre'] 

df.head()

,OTUNumber,OTUConTaxonomy,door_in_1,door_in_2,faucet_handle_1,faucet_handle_2,sink_floor_1,sink_floor_2,soap_dispenser_1,stall_in_1,...,toilet_flush_handle_1,toilet_flush_handle_2,toilet_seat_1,toilet_seat_2,regne,embranchement,classe,ordre,famille,genre
0,Otu00001,Bacteria(100);Actinobacteria(100);Actinobacter...,1381,1032,335,577,38,36,1115,598,...,13,329,76,156,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Propionibacteriaceae,Propionibacterium
1,Otu00002,Bacteria(100);Proteobacteria(100);Gammaproteob...,20,17,1686,3,411,564,128,9,...,0,25,258,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Enhydrobacter
2,Otu00003,Bacteria(100);Actinobacteria(100);Actinobacter...,8,47,6,418,15,8,84,45,...,3,283,41,1059,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium
3,Otu00004,Bacteria(100);Actinobacteria(100);Actinobacter...,6,27,5,6,452,275,63,0,...,9,51,5,3,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Micrococcaceae,Arthrobacter
4,Otu00005,Bacteria(100);Actinobacteria(100);Actinobacter...,0,29,0,648,2,8,3,32,...,0,206,7,95,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium


In [82]:
# ACP des differentes echantillons 
import plotly.graph_objects as go
import matplotlib
from plotly.validators.scatter.marker import SymbolValidator

matrice=df.groupby(by="genre").sum()
samples = list(matrice.columns.values)


#normalisation de la matrice transposé regrouper par embranchement 
matriceT = matrice.transpose()
matriceTN = StandardScaler().fit_transform(matriceT)

#Calcul de l'ACP avec sklearn en conservant 2 var
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(matriceTN)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

#Aout de differentes colonnes
principalDf['echantillon']=samples

type_ech = list(map(lambda x:x[-1], samples) )
principalDf['type']= type_ech

group_ech= list(map(lambda x:x[:-2], samples))
principalDf['groupe']= group_ech


color = {'door_in': 'greenyellow',
         'faucet_handle': 'gold',
         'sink_floor': 'lightblue',
         'soap_dispenser': 'blue',
         'stall_in': 'black',
         'toilet_floor': 'darkblue',
         'toilet_flush_handle': 'darkgreen',
         'toilet_seat': 'rebeccapurple'}

print( principalDf)
fig = go.Figure()
for g in principalDf['groupe'].unique():
    dfp = principalDf[principalDf['groupe']==g]
    fig.add_traces(go.Scatter(x=dfp['principal component 1'], 
                              y=dfp['principal component 2'], 
                              mode='markers',
                              marker_symbol=principalDf['type'],
                              name=g,
                              text=dfp['echantillon'],
                              marker_size=8,
                              marker = dict(color=color[g])
                             ))

fig.update_layout(title="ACP sur les echantilons et les bacteries presentes" )

fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="right",
    x=0.99,
))

fig.show()

    principal component 1  principal component 2            echantillon type  \
0               -7.423837               2.491412              door_in_1    1   
1               -4.589642               1.049099              door_in_2    2   
2               -5.447359               1.231082        faucet_handle_1    1   
3               -6.007490               1.279878        faucet_handle_2    2   
4                8.310246              -7.227117           sink_floor_1    1   
5                8.482028             -13.440246           sink_floor_2    2   
6               -5.714408               2.062533       soap_dispenser_1    1   
7               -5.970839               2.746583             stall_in_1    1   
8                9.374903              -6.324000         toilet_floor_1    1   
9                6.154425              -7.000304         toilet_floor_2    2   
10              19.920189              18.505469  toilet_flush_handle_1    1   
11              -5.024728               

In [42]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data


app = JupyterDash(__name__)

choix = ['regne','embranchement','classe','ordre', 'famille', 'genre']

app.layout = html.Div([
    html.H1("Utilisation de JupyterDash afin de visualiser les differents niveaux taxonomiques"),
    html.P("Blabla sur diversité"),
    html.Label([
        "Niveau taxonomique choisi",
        dcc.Dropdown(
            id="dropdown",
            options=[{"label": x, "value": x} for x in choix],
            value=choix[2],
            clearable=False )
    ]),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown", "value")])

#Definition du graphique a afficher
def update_bar_chart(value):
    fig = px.bar(df.groupby(by=value).sum().transpose(),           
             y=df.groupby(by=value).sum().index.values, 
             x=df.groupby(by=value).sum().columns.values).update_layout(showlegend=False)
    return fig

# Lancement de l'app en ligne 
app.run_server(mode='inline')

In [5]:
###OBJECTIF: Investigation sur l'apha diversité des échantillons
##http://www.countrysideinfo.co.uk/simpsons.htm
##http://www.evolution.unibas.ch/walser/bacteria_community_analysis/2015-02-10_MBM_tutorial_combined.pdf
########
#With this index, 0 represents infinite diversity and 1, no diversity. That is, the bigger the value of D, 
#the lower the diversity. This is neither intuitive nor logical, so to get over this problem, 
#D is often subtracted from 1 to give:
#Simpson's Index of Diversity 1 - D
#The value of this index also ranges between 0 and 1, but now, the greater the value, 
#the greater the sample diversity. This makes more sense. In this case, the index represents the probability 
#that two individuals randomly selected from a sample will belong to different species
#######

matrice=df.groupby(by="genre").sum()
#print(matrice)

#matrice.div(matrice.sum(axis=0))
#sum(matrice.div(matrice.sum(axis=0)).pow(1./2))
D= matrice.applymap(lambda x: x*(x-1)).sum().div(matrice.sum(axis=0)*(matrice.sum(axis=0)-1))
sympson_index= 1-D
sympson_index=sympson_index.sort_values()
indexNamesArr = sympson_index.index.values
indexNamesArr


mes_couleurs = ['greenyellow',
                'pink', 
                'chartreuse','pink',  
                'darkmagenta', 
                'gold','darkmagenta','tomato',
                'fuchsia','greenyellow','fuchsia',
                'tomato','rebeccapurple','chartreuse']

fig = go.Figure(go.Bar(
           y=sympson_index,
            x=indexNamesArr,
            marker=dict(
            color=mes_couleurs)))

fig.update_layout(title="Diversité ???" )

fig.show()

In [6]:
#Objectif : Génération d'un graphine sunburst interactif avec ipywidget
# Pouvoir visualiser la repartion des bacteries en sunburst dans les differents echantillons

#https://plotly.com/python/figurewidget-app/
#https://ipywidgets.readthedocs.io/en/latest/user_install.html
matrice=df.groupby(by="embranchement").sum()

embranchement = list(matrice.index.values)
samples = list(matrice.columns.values)

#definition d'un menu deroulant contenant les noms des differents echantillons 
choice = widgets.Dropdown(
    options=list(samples),
    value='door_in_1',
    description='Echantillon : ',
)
            
# Creation du widget contenant le graphique avec plolty express
g = go.FigureWidget(px.sunburst(df,
            path=['embranchement', 'genre'],
            values=choice.value,
            color_discrete_sequence=px.colors.sequential.Viridis,
            title="Repartition des bacteries dans l'echantillon "+choice.value))

#Fonction appelé par un changement de valeur dans le widget echantillon
def response(change):
    #Creation d'un 2eme graphiques 
    g2 = px.sunburst(df,
            path=['embranchement', 'genre'],
            values=choice.value,
            color_discrete_sequence=px.colors.sequential.Viridis,
            title="Répartition des bacteries dans l'echantillon "+choice.value)
    with g.batch_update():
            g.data[0].hovertemplate = g2.data[0].hovertemplate
            g.data[0].ids = g2.data[0].ids
            #g.data[0].labels = g2.data[0].labels
            g.data[0].parents = g2.data[0].parents
            g.data[0].values = g2.data[0].values
            g.layout.title = "Répartition des bacteries dans l'echantillon "+choice.value

# Objservation d'un changement de valeur
choice.observe(response , names='value')              

# Affichage des widgets
widgets.VBox([choice,g])

In [7]:
#Generation d'une heatmap 

#On regroupe les données par embrancchements
matrice=df.groupby(by="embranchement").sum()

#On recuperer les noms des indexs et des columns dans des listes
embranchement = matrice.index.values
samples = list(matrice.columns.values)

#Generation de la heatmap avec plotly
fig = go.Figure(data=go.Heatmap(
                   z=matrice,
                   x=samples,
                   y=embranchement,
                   hoverongaps = False))
fig.show()

In [8]:
#Objectif : Visualisation de la repartition des differents embranchements 
#           au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="embranchement").sum()


embranchement = matrice.index.values
samples = list(matrice.columns.values)

#Generation du barplot avec plotly express
fig = px.bar(matrice.transpose(), 
             x=samples, 
             y=embranchement, 
             color_discrete_sequence=px.colors.sequential.Viridis, 
             title="Répartition des differents embranchements")
fig.show()

In [9]:
#Objectif : Visualisation de la repartition principale bacteries 
#         au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="genre").sum()


#filtre les bacteries peu presentes

#matrice[ matrice.loc[ : ] < 50 ] = 0

#transformation en pourcentage 
#for col in matrice.columns.values :
#    matrice[col] = (matrice[col] / matrice[col].sum()) * 100


bacterie = matrice.index.values
samples = list(matrice.columns.values)



fig = px.bar(matrice.transpose(),           
             x=samples, 
             y=bacterie,      
             color_discrete_sequence=px.colors.sequential.Viridis,
             title="Répartition des principaux genres bacteriens dans les echantillons")
fig.show()


#On peut observer que l'echantillon "toilet_flush_handle_1" a beaucoup de bacterie unclassified

In [10]:
#Objectif : Visualisation de la repartition des principale famille de bacterie 
#     au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="famille").sum()

famille = matrice.index.values
samples = list(matrice.columns.values)



fig = px.bar(matrice.transpose(),           
             x=samples, 
             y=famille,        
             color_discrete_sequence=px.colors.sequential.Viridis,
             title="Répartition des principales familles bacteriennes dans les echantillons")
fig.show()

In [11]:
#Objectif : Visualisation de la repartition des differentes souches 
#            au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="genre").sum()

embranchement = list(matrice.index.values)
samples = list(matrice.columns.values)


fig = px.bar(matrice, 
            x=embranchement,
            y=samples, 
             
             color_discrete_sequence=px.colors.sequential.Viridis, 
             title="Répartition des differents genres bacteriens")
fig.show()

In [12]:
#Objectif : Visualisation de la variance des differents embranchements

matrice=df.groupby(by="embranchement").sum()
embranchement = list(matrice.index.values)
samples = list(matrice.columns.values)


fig = px.box(matrice, 
             x=embranchement,
             y=samples,
             color=embranchement,
             title="Variance de la présences des differentes embranchements dans les echntillons")
fig.update_layout(showlegend=False)
fig.show()

In [24]:
# ACP des differentes echantillons 

matrice=df.groupby(by="genre").sum()


samples = list(matrice.columns.values)


#normalisation de la matrice transposé regrouper par embranchement 
matriceT = matrice.transpose()
matriceTN = StandardScaler().fit_transform(matriceT)

#Calcul de l'ACP avec sklearn en conservant 2 var
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(matriceTN)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

principalDf['sample']=samples

fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', color='sample', 
                 title="ACP sur les echantilons et les bacteries presentes")
fig.show()